In [2]:
!pip install gradio

In [18]:
import torch
from torchvision import transforms

# 圖片預處理步驟（一定要有）
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [19]:
import torch
import torch.nn as nn
import pandas as pd

# 設定運算設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 載入說明表與類別名稱
df = pd.read_csv("C:/Users/echo6/Downloads/Food/food_info.csv")
food_dict = dict(zip(df['food'], df['description']))
class_names = sorted(list(food_dict.keys()))

# LeNet 模型定義
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.AvgPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 載入模型
model = LeNet5(num_classes=len(class_names)).to(device)
model.load_state_dict(torch.load("model_food.pth", map_location=device))
model.eval()

def predict_with_gradio(image):
    try:
        image = image.convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)
            top3 = torch.topk(probs, 3)

            result_text = "📊 預測前 3 名：\n"
            for i in range(3):
                idx = top3.indices[0][i].item()
                label = class_names[idx]
                prob = top3.values[0][i].item()
                result_text += f"Top {i+1}: {label}（信心值：{prob:.2%}）\n"

            # 最可能的那一個
            main_idx = top3.indices[0][0].item()
            main_label = class_names[main_idx]
            description = food_dict.get(main_label, "查無資料")

            result_text += f"\n🍽️ 預測食物（最可能）：{main_label}\n"
            result_text += f"📘 食物說明：\n{description}"
            return result_text

    except Exception as e:
        return f"❌ 分析時發生錯誤：{str(e)}"



In [20]:
import gradio as gr

interface = gr.Interface(
    fn=predict_with_gradio,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title=" AI 美食分類與說明系統",
    description="上傳一張食物圖片，系統會辨識食物類別並顯示中文說明"
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
